# **ДЗ №2 STEM. Алексеев Виталий**

## **0 Импорт библиотек**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/Raven-ES1617/cu/refs/heads/main/science_studio/whole_data_practice_4.csv")
df

,RAJ2000,DEJ2000,Vmag,e_Vmag,Bmag,e_Bmag,gpmag,e_gpmag,rpmag,e_rpmag,ipmag,e_ipmag,fuv_mag,nuv_mag,err,present,min_mag,max_mag
0,168.580444,1.051704,11.143,0.027,11.592,0.025,13.105,3.575,10.993,0.165,11.012,0.052,18.347,14.734,3.579373,0,13.29,13.41
1,141.300348,3.499374,11.564,0.056,12.160,0.084,11.716,0.123,13.713,2.663,11.033,0.400,23.511,16.393,2.697571,0,13.29,13.41
2,129.795551,18.175921,10.179,0.038,10.731,0.013,11.355,1.902,10.829,1.883,10.018,0.010,20.863,14.930,2.676753,0,13.29,13.41
3,118.424259,21.332130,11.203,0.023,11.553,0.033,11.374,0.050,11.141,0.040,12.418,2.615,18.715,14.744,2.616093,0,13.29,13.41
4,62.954121,-5.236932,10.699,0.038,11.382,0.094,11.896,2.567,10.502,0.070,10.418,0.070,22.283,15.858,2.570908,0,13.29,13.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56293,155.694165,1.603857,11.583,0.014,12.064,0.006,11.763,0.005,11.459,0.005,11.326,0.006,22.445,15.643,0.017833,0,13.11,0.59
56294,245.021189,12.281058,11.780,0.006,12.275,0.006,11.974,0.006,11.689,0.006,11.574,0.012,22.605,16.135,0.016971,0,13.11,0.59
56295,26.220662,0.868477,12.502,0.005,12.952,0.005,12.676,0.009,12.353,0.011,12.256,0.006,22.894,16.442,0.016971,0,13.11,0.59
56296,115.828020,19.550841,11.881,0.006,12.795,0.008,12.302,0.006,11.566,0.008,11.261,0.006,22.743,17.458,0.015362,0,14.49,14.53


## **1 Валидация и исправление min/max**

In [3]:
df_stage_1 = df.copy()
mask = df_stage_1["min_mag"] > df_stage_1["max_mag"]
df_stage_0 = df.loc[mask, ["min_mag", "max_mag"]]
df_stage_1.loc[mask, ["min_mag", "max_mag"]] = df_stage_1.loc[mask, ["max_mag", "min_mag"]].values

print(f"Кол-во изменений: {mask.count()}")
df_stage_0.join(
    df_stage_1.loc[mask, ['min_mag', 'max_mag']].copy(),
    lsuffix='_before', rsuffix='_after'
).head(10)

Кол-во изменений: 56298


,min_mag_before,max_mag_before,min_mag_after,max_mag_after
6,9.700,0.54,0.54,9.700
7,9.700,0.54,0.54,9.700
8,9.700,0.54,0.54,9.700
9,9.700,0.54,0.54,9.700
10,9.700,0.54,0.54,9.700
11,9.700,0.54,0.54,9.700
12,9.700,0.54,0.54,9.700
13,9.700,0.54,0.54,9.700
14,9.700,0.54,0.54,9.700
36,15.178,1.12,1.12,15.178


In [4]:
df = df_stage_1

Стоить заметить, что даже такие простые ошибки, такие как перепутанные min и max, могут сильно повлиять на дальнейший анализ данных. В данном случае, если бы не была проведена валидация ($\approx 56\;000$ значений), то дальнейшие шаги по нормализации и поиску выбросов могли бы быть некорректными.

## **2 Инверсионно-взвешенное среднее оптической величины и сравнение групп**

In [5]:
mag_columns = ["Vmag", "Bmag", "gpmag", "rpmag", "ipmag"]
error_columns = [f"e_{mag}" for mag in mag_columns]

df_stage_2 = df.copy()


def find_inv_weighted_mean(row):
    mags = row.loc[mag_columns].values
    errors = row[error_columns].values

    valid = (~np.isnan(mags)) & (~np.isnan(errors)) & (errors > 0)
    if not valid.any():
        return pd.Series([np.nan, np.nan])

    mags, errors = mags[valid], errors[valid]

    w = 1 / errors ** 2
    m_mean = np.sum(w * mags) / np.sum(w)
    e_mean = np.sqrt(1 / np.sum(w))

    return pd.Series([m_mean, e_mean])


df_stage_2[["mean_opt_mag", "e_mean_opt_mag"]] = df_stage_2.apply(
    find_inv_weighted_mean, axis=1
)
df_stage_2.groupby("present").agg(
    n_objects=("Vmag", "count"),
    mean_opt_mag=("mean_opt_mag", "mean"),
    median_opt_mag=("mean_opt_mag", "median"),
    mean_opt_error_mag=("e_mean_opt_mag", "mean"),
    median_opt_error_mag=("e_mean_opt_mag", "median"),
    freq_low_error=('e_mean_opt_mag', lambda x: (x < 0.01).mean())
    # Порог необходимо установить достаточно малым (~1%), чтобы данные, имеющие ошибку, близкую к нулю, что не может быть правдой, не влияли на результат
)

,n_objects,mean_opt_mag,median_opt_mag,mean_opt_error_mag,median_opt_error_mag,freq_low_error
present,,,,,,
0,50388,13.052566,12.841676,0.016528,0.014608,0.251191
1,5910,13.321139,13.447680,0.025842,0.017307,0.181387


In [6]:
result_df = pd.DataFrame()
for present in 0, 1:
    result_df = pd.concat(
        [
            result_df,
            df_stage_2.sort_values("mean_opt_mag", na_position="last").loc[
                df_stage_2["present"] == present,
                ["mean_opt_mag", "e_mean_opt_mag", "present"]
            ].head(10)
        ], axis=0
    )
result_df

,mean_opt_mag,e_mean_opt_mag,present
2400,5.395984,0.051343,0
3181,5.710773,0.006966,0
10708,5.804261,0.002977,0
55197,6.055452,0.001680,0
1000,6.158925,0.067695,0
5881,6.231023,0.014217,0
40972,6.283910,0.006140,0
55626,6.549199,0.006648,0
13068,6.550119,0.015705,0
55276,6.628873,0.002642,0


Исходя из полученных данных, можно сделать вывод, что переменные звезды имеют в срнедем большую оптическую величину ($\approx +2\%$), и большую ошибку измерения ($\approx +5.3\%$), что связано в первую очередь с тем, что во время измерения звезда меняет свою яркость и вследствие этого добавочная якрость идет в счет ошибки.

## **3 Амплитуды и сравнение групп (без статистической проверки)**

In [7]:
df_stage_3 = df.copy()

df_stage_3["obs_amp"] = df_stage_3["max_mag"] - df_stage_3["min_mag"]
df_stage_3.groupby("present").agg(
    n_objects=("obs_amp", "count"),
    mean_obs_amp=("obs_amp", "mean"),
    median_obs_amp=("obs_amp", "median"),
    percentile=('obs_amp', lambda x: np.percentile(x, 0.75)),
    freq_amp=('obs_amp', lambda x: (x > 0.1).mean())
)

,n_objects,mean_obs_amp,median_obs_amp,percentile,freq_amp
present,,,,,
0,50388,3.373688,0.05,0.02,0.374752
1,5910,3.694575,0.06,0.02,0.420981


In [8]:
df_stage_3.groupby("present").agg(
    first_bin=("obs_amp", lambda x: x.between(0, 0.05).sum()),
    second_bin=("obs_amp", lambda x: x.between(0.05, 0.2).sum()),
    third_bin=("obs_amp", lambda x: (x > 0.2).sum()),
)

,first_bin,second_bin,third_bin
present,,,
0,25456,8581,16351
1,2761,995,2154


In [9]:
result_df = pd.DataFrame()
for present in 0, 1:
    result_df = pd.concat(
        [
            result_df,
            df_stage_3.sort_values("obs_amp", na_position="first", ascending=False).loc[
                df_stage_2["present"] == present,
                ["Vmag", "obs_amp", "present"]
            ].head(10)
        ], axis=0
    )
result_df

,Vmag,obs_amp,present
12134,11.435,16.350,0
12147,16.375,16.350,0
12158,12.421,16.350,0
12157,10.315,16.350,0
12156,10.649,16.350,0
12155,12.719,16.350,0
12153,13.602,16.350,0
12152,12.947,16.350,0
12151,14.795,16.350,0
12150,16.119,16.350,0


Исходя из полученных распределений, можно заметить, что переменные звезды имеют в среднем большую амплитуду наблюдаемой величины ($\approx +0.9\%$). Из распределения амплитуд имеем, что большинство переменных звезд имеют либо слишком малую амплитуду, либо слишком большую. Отсюда возникает гипотеза о разделении на классы согласно этому параметру.

## **4 Цветовые индексы и частота экстремов**

In [10]:
df_stage_4 = df.copy()
color_indices = {
    "B-V": ("Bmag", "Vmag", (-0.3, 2.2)),
    "g-r": ("gpmag", "rpmag", (-0.5, 1.8)),
    "r-i": ("rpmag", "ipmag", (-0.2, 2.0)),
}
for color, mags in color_indices.items():
    index = df_stage_4[mags[0]] - df_stage_4[mags[1]]
    error = np.sqrt(
        df_stage_4[f"e_{mags[0]}"] ** 2 + df_stage_4[f"e_{mags[1]}"] ** 2)
    df_stage_4[color] = index
    df_stage_4[f"e_{color}"] = error
    df_stage_4[f"is_extreme_{color}"] = (
            (np.abs(index) > 3 * error) & (~index.between(*mags[2]))
    ).astype(int)
# Пределы необходимо искать используя модель абсолютно черного тела,
# после найти для каждого фильтра интегральный поток и после построить
# функцию от температуры, в коде приведены уже посчитанные и указанные
# в каталогах значения
df_stage_4

,RAJ2000,DEJ2000,Vmag,e_Vmag,Bmag,e_Bmag,gpmag,e_gpmag,rpmag,e_rpmag,...,max_mag,B-V,e_B-V,is_extreme_B-V,g-r,e_g-r,is_extreme_g-r,r-i,e_r-i,is_extreme_r-i
0,168.580444,1.051704,11.143,0.027,11.592,0.025,13.105,3.575,10.993,0.165,...,13.41,0.449,0.036797,0,2.112,3.578806,0,-0.019,0.173000,0
1,141.300348,3.499374,11.564,0.056,12.160,0.084,11.716,0.123,13.713,2.663,...,13.41,0.596,0.100955,0,-1.997,2.665839,0,2.680,2.692874,0
2,129.795551,18.175921,10.179,0.038,10.731,0.013,11.355,1.902,10.829,1.883,...,13.41,0.552,0.040162,0,0.526,2.676433,0,0.811,1.883027,0
3,118.424259,21.332130,11.203,0.023,11.553,0.033,11.374,0.050,11.141,0.040,...,13.41,0.350,0.040224,0,0.233,0.064031,0,-1.277,2.615306,0
4,62.954121,-5.236932,10.699,0.038,11.382,0.094,11.896,2.567,10.502,0.070,...,13.41,0.683,0.101390,0,1.394,2.567954,0,0.084,0.098995,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56293,155.694165,1.603857,11.583,0.014,12.064,0.006,11.763,0.005,11.459,0.005,...,13.11,0.481,0.015232,0,0.304,0.007071,0,0.133,0.007810,0
56294,245.021189,12.281058,11.780,0.006,12.275,0.006,11.974,0.006,11.689,0.006,...,13.11,0.495,0.008485,0,0.285,0.008485,0,0.115,0.013416,0
56295,26.220662,0.868477,12.502,0.005,12.952,0.005,12.676,0.009,12.353,0.011,...,13.11,0.450,0.007071,0,0.323,0.014213,0,0.097,0.012530,0
56296,115.828020,19.550841,11.881,0.006,12.795,0.008,12.302,0.006,11.566,0.008,...,14.53,0.914,0.010000,0,0.736,0.010000,0,0.305,0.010000,0


In [11]:
df_stage_4_indexes_stats = df_stage_4.groupby("present").agg(
    B_V_count=("is_extreme_B-V", lambda x: (x == 1).sum()),
    g_r_count=("is_extreme_g-r", lambda x: (x == 1).sum()),
    r_i_count=("is_extreme_r-i", lambda x: (x == 1).sum()),
    B_V_freq=("is_extreme_B-V", lambda x: (x == 1).mean()),
    g_r_freq=("is_extreme_g-r", lambda x: (x == 1).mean()),
    r_i_freq=("is_extreme_r-i", lambda x: (x == 1).mean()),
)
df_stage_4_indexes_stats

,B_V_count,g_r_count,r_i_count,B_V_freq,g_r_freq,r_i_freq
present,,,,,,
0,1,1,96,0.000020,0.000020,0.001905
1,1,3,15,0.000169,0.000508,0.002538


In [12]:
df_stage_4.loc[
    df_stage_4[f"is_extreme_B-V"] == 1, ["B-V", "e_B-V", "present"]
].head(10)

,B-V,e_B-V,present
13607,-0.329,0.070605,1
22061,-0.334,0.079756,0


In [13]:
df_stage_4.loc[
    df_stage_4[f"is_extreme_g-r"] == 1, ["g-r", "e_g-r", "present"]
].head(10)

,g-r,e_g-r,present
742,2.102,0.662091,1
1667,1.823,0.447001,0
13331,-0.542,0.081049,1
13607,-0.540,0.163355,1


In [14]:
df_stage_4.loc[
    df_stage_4[f"is_extreme_r-i"] == 1, ["r-i", "e_r-i", "present"]
].head(10)

,r-i,e_r-i,present
742,2.537,0.525385,1
2644,-0.308,0.087693,0
3936,-0.364,0.096897,0
4801,2.113,0.239109,1
8052,-0.490,0.161245,0
9305,-0.399,0.122430,0
10062,-0.278,0.044385,0
12762,-0.271,0.084865,0
13945,-0.238,0.072111,0
14417,-0.263,0.038833,1


In [15]:
for color in "B_V_freq", "g_r_freq", "r_i_freq":
    print(
        f"R({color}) = {df_stage_4_indexes_stats.loc[0, color] / df_stage_4_indexes_stats.loc[1, color]:.2f}"
    )

R(B_V_freq) = 0.12
R(g_r_freq) = 0.04
R(r_i_freq) = 0.75


## **5 Согласованность по фильтрам: редуцированная $\chi^2$**

In [ ]:
df_stage_5 = df_stage_2.copy()

def compute_reduced_chi2(row):
    mags = row[mag_columns].values
    errs = row[error_columns].values
    mean_mag = row["mean_opt_mag"]

    valid = (~np.isnan(mags)) & (~np.isnan(errs)) & (errs > 0)
    n_used = valid.sum()
    if n_used <= 1 or np.isnan(mean_mag):
        return np.nan

    mags, errs = mags[valid], errs[valid]
    chi2 = np.sum(((mags - mean_mag) ** 2) / (errs ** 2))
    chi2_red = chi2 / (n_used - 1)
    return pd.Series([n_used, chi2_red])


df_stage_5[["n_used_filters", "chi2_red"]] = df_stage_5.apply(compute_reduced_chi2, axis=1)

In [138]:
df_stage_5.groupby("present").agg(
    n_objects=("chi2_red", "count"),
    mean_chi2_red=("chi2_red", "mean"),
    median_chi2_red=("chi2_red", "median"),
    frac_high_chi2=("chi2_red", lambda x: (x > 3).mean())
)

,n_objects,mean_chi2_red,median_chi2_red,frac_high_chi2
present,,,,
0,50388,138.498376,63.423024,0.921688
1,5910,143.117056,39.801658,0.772081


In [136]:
df_stage_5.sort_values(
    "chi2_red", ascending=False
).groupby("present").head(10)[
    ["chi2_red", "n_used_filters", "present"]
]

,chi2_red,n_used_filters,present
55197,37639.347005,5.0,0
513,33089.076163,5.0,0
56249,28248.103260,5.0,0
56255,24823.975422,5.0,1
54610,23193.017123,5.0,0
24569,21933.775018,5.0,0
55276,20827.529086,5.0,0
41600,19127.507051,5.0,0
46840,13044.204683,5.0,1
56112,12950.485982,5.0,0


Можем, заметить, что переменные звезды имеют в среднем большую редуцированную $\chi^2$ ($\approx +3.6\%$), что связано с тем, что величина звезды меняется, и измерения в разных фильтрах могут быть сделаны в разное время, что приводит к большей расходимости измерений. Однако медианы соответствующих значений сильной отличаются от средних, что говорит о наличии сильного скошенного распределения. Также можно заметить, что доля объектов с $\chi^2_{red} > 3$ среди не переменных звезд на 10% больше, чем среди переменных.

## **6 UV-эксцессы и их связь с переменностью**

In [149]:
df_stage_6 = df_stage_5.copy()

mask_valid_uv = df_stage_6[["fuv_mag", "nuv_mag"]].notna().all(axis=1)
df_stage_6.loc[mask_valid_uv, "fuv_minus_nuv"] = (
    df_stage_6.loc[mask_valid_uv, "fuv_mag"] - df_stage_6.loc[mask_valid_uv, "nuv_mag"]
)

mu_pop = df_stage_6["fuv_minus_nuv"].mean(skipna=True)
sigma_pop = df_stage_6["fuv_minus_nuv"].std(skipna=True)

df_stage_6["z_uv"] = (df_stage_6["fuv_minus_nuv"] - mu_pop) / sigma_pop

df_stage_6["is_uv_excess"] = (df_stage_6["z_uv"] < -3).astype(int)

df_stage_6.groupby("present").agg(
    n_objects=("is_uv_excess", "sum"),
    frac_uv_excess=("is_uv_excess", "mean")
)

,n_objects,frac_uv_excess
present,,
0,7,0.000139
1,0,0.000000


In [143]:
print(f"Доля UV-эксцессов среди переменных звезд: {df_stage_6.loc[df_stage_6["is_uv_excess"], "present"].mean()}")

Доля UV-эксцессов среди переменных звезд: 0.0


In [144]:
df_stage_6.loc[
    df_stage_6["is_uv_excess"]
].sort_values("z_uv").head(10)[["fuv_minus_nuv", "z_uv", "present"]]

,fuv_minus_nuv,z_uv,present
43892,-1.496,-3.189523,0
19917,-1.296,-3.087661,0
34607,-1.294,-3.086642,0
47621,-1.240,-3.059139,0
10298,-1.234,-3.056083,0
6491,-1.186,-3.031636,0
23308,-1.153,-3.014829,0


Таким образом, мы выяснили, что переменные звезды не обладают эксцессом популяции, более того, $z$-оценка меньше 3 -- слишком сильное условие, потому как всего 7 объектов удовлетворяют данному условию.